# 不同物种同源基因转换

In [2]:
library(biomaRt)
library(Seurat)

Attaching SeuratObject



### 查看特定 'ensembl' 数据库的数据集

In [20]:
ensembl <- useMart("ensembl")
DBlist = listDatasets(ensembl)
dim(DBlist)
head(DBlist, 5)

[1] 215   3

,dataset,description,version
,<I<chr>>,<I<chr>>,<I<chr>>
1,abrachyrhynchus_gene_ensembl,Pink-footed goose genes (ASM259213v1),ASM259213v1
2,acalliptera_gene_ensembl,Eastern happy genes (fAstCal1.2),fAstCal1.2
3,acarolinensis_gene_ensembl,Green anole genes (AnoCar2.0v2),AnoCar2.0v2
4,acchrysaetos_gene_ensembl,Golden eagle genes (bAquChr1.2),bAquChr1.2
5,acitrinellus_gene_ensembl,Midas cichlid genes (Midas_v5),Midas_v5


### 选择人、猴数据库

In [61]:
ensembl.macaque <- useMart("ensembl", dataset = "mmulatta_gene_ensembl", host = "http://dec2021.archive.ensembl.org/")

Ensembl site unresponsive, trying uswest mirror



In [62]:
ensembl.human <- useMart("ensembl", dataset = "hsapiens_gene_ensembl", host = "http://dec2021.archive.ensembl.org/")

Ensembl site unresponsive, trying uswest mirror



#### ref
* https://www.jianshu.com/p/abf5138e3757

## listAttributes函数查看可选择输出的类型

In [64]:
listAttributes(ensembl.macaque)

name,description,page
<chr>,<chr>,<chr>
ensembl_gene_id,Gene stable ID,feature_page
ensembl_gene_id_version,Gene stable ID version,feature_page
ensembl_transcript_id,Transcript stable ID,feature_page
ensembl_transcript_id_version,Transcript stable ID version,feature_page
ensembl_peptide_id,Protein stable ID,feature_page
ensembl_peptide_id_version,Protein stable ID version,feature_page
ensembl_exon_id,Exon stable ID,feature_page
description,Gene description,feature_page
chromosome_name,Chromosome/scaffold name,feature_page


### 读取待转换的猴基因列表

In [31]:
rds <- readRDS("~/Documents/macaque_test.rds")
macaque_features <- rownames(rds@assays$RNA@counts)

In [51]:
macaque_features[24100:24150]

[1] "ENSMMUG00000063905" "ENSMMUG00000063949" "ENSMMUG00000064080"
 [4] "ENSMMUG00000064558" "ENSMMUG00000064560" "ENSMMUG00000064584"
 [7] "ENSMMUG00000064661" "ENSMMUG00000064689" "ENSMMUG00000064858"
[10] "ENSMMUG00000065031" "ENSMMUG00000065132" "ENSMMUG00000065141"
[13] "ERVV.2"             "ETV3L"              "FCRL4"             
[16] "FGA"                "G6PC2"              "GABRR2"            
[19] "GC"                 "GLT6D1"             "GPR15"             
[22] "KCNG4"              "KRT20"              "LDLRAD2"           
[25] "LINC00514"          "MAGEA8"             "MEFV"              
[28] "MPC1L"              "MUC20"              "NRTN"              
[31] "ODF4"               "OR12D2"             "OR4K13"            
[34] "OR8H3"              "PDZD3"              "PRR20G"            
[37] "PRSS54"             "RDH8"               "RGS21"             
[40] "SDS"                "SFN"                "SIRPG"             
[43] "SLC5A12"            "SNORD83A"           "SPANXN4"           
[46] "SPATA25"            "SPTLC3"             "TEX36"             
[49] "TFF1"               "TGM7"               "UBQLNL"

In [66]:
macaque_features_id_l <- grepl("ENSMMUG", macaque_features)

macaque_features_symbol <- macaque_features[!macaque_features_id_l]
macaque_features_id <- macaque_features[macaque_features_id_l]

### 分别转换猴gene列表里的gene_symbol和gene_ID为人中的同源基因

In [65]:
macaca2human.gene_symbol <- getLDS(attributes = c("ensembl_gene_id", "external_gene_name"),
                filters = c("external_gene_name"),
                values = macaque_features_symbol, 
                mart = ensembl.macaque,
                attributesL = c("ensembl_gene_id", "external_gene_name"),
                martL = ensembl.human,
                uniqueRows = T)

head(macaca2human.gene_symbol,2)

,Gene.stable.ID,Gene.name,Gene.stable.ID.1,Gene.name.1
,<chr>,<chr>,<chr>,<chr>
1,ENSMMUG00000065356,ND2,ENSG00000198763,MT-ND2
2,ENSMMUG00000065375,ATP6,ENSG00000198899,MT-ATP6


In [67]:
macaca2human.gene_id <- getLDS(attributes = c("ensembl_gene_id", "external_gene_name"),
                filters = c("ensembl_gene_id"),
                values = macaque_features_id, 
                mart = ensembl.macaque,
                attributesL = c("ensembl_gene_id", "external_gene_name"),
                martL = ensembl.human,
                uniqueRows = T)

head(macaca2human.gene_id, 2)

,Gene.stable.ID,Gene.name,Gene.stable.ID.1,Gene.name.1
,<chr>,<chr>,<chr>,<chr>
1,ENSMMUG00000056709,,ENSG00000198695,MT-ND6
2,ENSMMUG00000063555,,ENSG00000198695,MT-ND6


#### 
* 去除未转换成功的gene
* 去除macaque 重复的基因

In [68]:
macaca2human.gene_id <- macaca2human.gene_id[macaca2human.gene_id$Gene.name.1 != "",]
dim(macaca2human.gene_id)
macaca2human.gene_symbol <- macaca2human.gene_symbol[macaca2human.gene_symbol$Gene.name.1 != "",]
dim(macaca2human.gene_symbol)


macaca2human.gene_id <- macaca2human.gene_id[duplicated(macaca2human.gene_id$Gene.stable.ID)==FALSE,]
dim(macaca2human.gene_id)
macaca2human.gene_symbol <- macaca2human.gene_symbol[duplicated(macaca2human.gene_symbol$Gene.name)==FALSE,]
dim(macaca2human.gene_symbol)

[1] 1457    4

[1] 17213     4

[1] 1005    4

[1] 15282     4

* 把macaque gene统一到Gene.name列
* 合并symbol和ID分别对应的human中的同源基因
* 去除多个macaque基因对应相同human gene的情况

In [69]:
macaca2human.gene_id$Gene.name <- macaca2human.gene_id$Gene.stable.ID
macaca2human <- rbind(macaca2human.gene_symbol, macaca2human.gene_id)
dim(macaca2human)

macaca2human <- macaca2human[duplicated(macaca2human$Gene.name.1)==FALSE,]
dim(macaca2human)

[1] 16287     4

[1] 15945     4

* 从rds中总的macaque_features中提取转换成human 同源基因成功的gene
* 对原rds表达矩阵做subset，并替换macaque gene为对应转换的human gene

In [72]:
macaca_features_df <- data.frame(macaque_features)
length(macaque_features)
dim(macaca_features_df)
row.names(macaca_features_df) <- macaque_features
dim(macaca_features_df)
macaca_features_df[macaca2human$Gene.name, "macaque_features"] <- macaca2human$Gene.name.1
dim(macaca_features_df)

[1] 24333

[1] 24333     1

[1] 24333     1

[1] 24333     1

In [73]:
macacaque_human_symbol <- subset(macaca_features_df, duplicated(macaca_features_df$macaque_features)==FALSE)

macaca_mat <- data.frame(rds@assays$RNA@counts)
macaca_mat <- macaca_mat[rownames(macacaque_human_symbol),]
rownames(macaca_mat) <- macacaque_human_symbol$macaque_features

# 文档地址
* https://github.com/seqyuan/cortex/Species_homologous_gene_conversion.ipynb